<a href="https://colab.research.google.com/github/maicon-reis/python_para_financas/blob/main/Carteira_Dividendos_Otimizada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalando a Bibliotecha PyPortifolioOpt
!pip install -q PyPortfolioOpt

In [ ]:
# bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_datareader import data as wb
import matplotlib
from scipy import stats
from datetime import date
from pypfopt.risk_models import CovarianceShrinkage, risk_matrix
from pypfopt import plotting, objective_functions, EfficientFrontier, expected_returns
from pypfopt import risk_models
from pypfopt.plotting import plot_efficient_frontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [ ]:
# carteira de Ativos
ativos = ['TAEE11.SA', 'BAZA3.SA', 'SANB3.SA', 'VIVT3.SA', 'SULA4.SA',
          'CPFE3.SA', 'CGRA4.SA', 'EGIE3.SA', 'MRVE3.SA', 'ITSA4.SA']

pesos = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

# Período
data_inicio = '2011-01-01'
data_fim = date.today().isoformat()

# criação do DataFrame
carteira = pd.DataFrame()

for t in ativos:
  carteira[t] = wb.DataReader(
      t, data_source='yahoo', start=data_inicio, end=data_fim
  )['Adj Close']

In [ ]:
carteira.head()

In [ ]:
carteira.info()

**Retorno Esperado e Modelo de Risco**

In [ ]:
# retorno esperado
mu = expected_returns.ema_historical_return(carteira, compounding=False, span=500)

# modelo de risco
S = CovarianceShrinkage(carteira, frequency=252).ledoit_wolf(shrinkage_target="single_factor")

**Otimizador de Média-Variância**

In [ ]:
# otimizador de miníma volatilidade
ef = EfficientFrontier(mu, S) # weight_bounds=(-1,1)

# ef.add_objective(objective_functions.L2_reg, gamma=0.1)
w = ef.min_volatility()

In [ ]:
cleaned_weights = ef.clean_weights(rounding=2)
cleaned_weights

**Desempenho Esperado**


In [ ]:
# desempenho esperado
ret_tangent, std_tangent, _ = ef.portfolio_performance(verbose=True)

**Fronteira Eficiente**

In [ ]:
from pypfopt import CLA

In [ ]:
portifolio_eficiente = CLA(mu, S)

In [ ]:
(retorno, volatilidade, peso) = portifolio_eficiente.efficient_frontier()

In [ ]:
fix, ax = plt.subplots( figsize=(8,6))
ax.scatter(volatilidade, retorno, s=40, c='g', marker='.', label='Fronteira Eficiente')
ax.scatter(std_tangent, ret_tangent, s=200, c='r', marker='*', label='Volatilidade Mínima')
ax.set_xlabel('Volatilidade')
ax.set_ylabel('Retorno Esperado')
ax.set_title('Fronteira Eficiente')
plt.show()

**Exemplo de Alocação de Patrimônio**

In [ ]:
# preços atuais
lp = get_latest_prices(carteira)

# Alocação por quantidade de um patrimônio total de R$ 10.000,00
da = DiscreteAllocation(w, lp, total_portfolio_value=10000)
alocacao, sobras = da.lp_portfolio()
alocacao